# Cross-validation : avantages et inconvénients

____

Liens : 
- [Cours OpenClassrooms](https://openclassrooms.com/fr/courses/4297211-evaluez-et-ameliorez-les-performances-dun-modele-de-machine-learning/4308241-mettez-en-place-un-cadre-de-validation-croisee)
- [Validation Croisée Yohan](https://fr.calameo.com/read/000003587247e52d9457e)
- [Vidéo Pierrot Machine Learnia](https://www.youtube.com/watch?v=VoyMOVfCSfc&t=1136s)
- [Dataset Iris sklearn](https://scikit-learn.org/stable/datasets/index.html)


In [1]:
import pandas as pd

### Ce qu'on faisait avant

Split du dataset en un ensemble d'entrainement et un ensemble pour estimer la performance de notre modèle
![train_test](img/train_test.png)

L'estimation de la perfomance de notre modele peut s'en retrouvé biaisée car elle dépent des données qui vont se retrouver dans l'ensemble de validation ( par exemple un ensemble de validation qui comporterait uniquement des observations d'une meme classe dans un problème de classification) 

### La cross validation

- La cross validation permet d'utiliser l'intégralité de notre jeu de donnée pour l'entrainement et la validation

- Fonctionnement : split du jeu de donnée en k parties ( folds )  à peu prés égales , tour à tour chaque partie est utilisée comme jeu de test , le reste sert à entrainer le modéle
- En général cette technique n'est pas utilisée en Deep Learning : 
    * Gros volume de données , il est peu problable d'avoir que des observations d'une classe dans un ensemble de validation
    * Si on fait par exemple 5 splits , on doit entrainer 5 modeles , ce qui peut prendre beaucoup de temps dans le cadre de l'entrainement d'un modele de deep learning

![k_folds](img/k_folds.png)


### Stratification

- Dans le cas d'un probleme de classification on s'assure de créer des k folds de sorte qu'elles contiennent à peu prés les memes proprotions d'exemple de chaque classe pour éviter de biaiser les résultats

![strat](img/strat.png)


### Dataset Iris

- Probleme de classification , prédire de quelle famille est une iris en fonction de 4 features ( longueur du pétale etc...)
- 3 familles différentes distribuées de maniere égale


In [2]:
from sklearn.datasets import load_iris

In [3]:
iris = load_iris()
X_train = iris.data
y_train = iris.target

#### KFold

- Bon pour les problemes de régression
- Mauvais quand les classes sont déséquilibrées

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [5]:
cv = KFold(5,random_state=0)
# random_state = seed ?
cross_val_score(KNeighborsClassifier(), X_train,y_train,cv=cv)
# score = accuracy ? possibilité de spécifier quelle metric on veut 

array([1.        , 1.        , 0.83333333, 0.93333333, 0.8       ])

#### Leave One Out
Cas "extréme " on on prend un nombre de folds égal au nombre d'observations du dataset , on verifie donc la perfomance du modele sur une seule observation à chaque fois

- Forte augmentation du temps de calcul pour un gros dataset
- Formation de jeux d'entrainement trés similaire entre eux ,  on aura quasiment le meme modéle sur chaque fold

In [6]:
from sklearn.model_selection import LeaveOneOut

In [7]:
cv = LeaveOneOut()
cross_val_score(KNeighborsClassifier(), X_train,y_train,cv=cv)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

#### Suffle Split

- split train/v comme on le faisait avant , mais on remélange et on refait un split n fois en précisant quel pourcentage des données est utilisé pour le test

In [11]:
from sklearn.model_selection import ShuffleSplit

In [12]:
cv = ShuffleSplit(4, test_size=0.2)
cross_val_score(KNeighborsClassifier(), X_train,y_train,cv=cv)

array([0.93333333, 0.96666667, 0.96666667, 0.96666667])

#### Stratified KFold


- commence par un tri des classes ( ex 1 ou 0 ) 
- en fonction du nombre de splits voulus , découpe de chaque classe en n split ( ex si on veut 4 splits , découpe de la classe 0 en 4 , découpe de la classe 1 en 4 )
- on regroupe ensuite 2 par 2 chaque split.
- On se retrouve ensuite avec 4 splits qui ont a peu prés la meme proprotion d'observation de la classe 1 et de la classe 0 

Pas trés clair à l'écrit , mais bien expliqué dans la vidéo vers 15:00 =)

In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
cv = StratifiedKFold(4)
cross_val_score(KNeighborsClassifier(), X_train,y_train,cv=cv)

array([0.96666667, 1.        , 0.93333333, 0.96666667, 1.        ])

###  est ce qu'on parle du GroupKFold ( vidéo )  pas sur de savoir l'expliquer =) ???